In [1]:
!pip install flash-attn --upgrade

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import torch
from flash_attn import flash_attn_func

In [26]:
import torch
batch_size=32
seq_len=4096
hidden_dim=256


Q = torch.randn(batch_size, seq_len, hidden_dim, dtype=torch.float16)
K = torch.randn(batch_size, seq_len, hidden_dim, dtype=torch.float16)
V = torch.randn(batch_size, seq_len, hidden_dim, dtype=torch.float16)


In [23]:
Q.shape

torch.Size([32, 20480, 256])

In [24]:
import torch.nn.functional as F
import math

## attention
def attention(q, k, v, mask=None, dropout=None):
    d_k = q.size(-1)
    scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, v), p_attn

## CPU ~5m


In [9]:
output, hidden_state = attention(Q, K, V)

## GPU pytorch ~0.1s


In [27]:
output, hidden_state = attention(Q.to('cuda'), K.to('cuda'), V.to('cuda'))

## flash attention

In [18]:
Q.shape, Q.unsqueeze(1).shape

(torch.Size([32, 2048, 256]), torch.Size([32, 1, 2048, 256]))

In [28]:
from flash_attn import flash_attn_func
output = flash_attn_func(Q.unsqueeze(1).to('cuda'), K.unsqueeze(1).to('cuda'), V.unsqueeze(1).to('cuda'))
output.shape

torch.Size([32, 1, 4096, 256])